# Preparation of Source Data
The “CSE-CIC-IDS2018” dataset consists of 10 CSV data files of total size 6.4GB. Each file captures a full day's worth of network events captured through 80 features. Majority of these files are approx 350kB of size, but one of the files is 4GB of size. 
This notebook will carry out the following "staging" activities of this data:
1. Read each file
2. Remove duplicated headers in the file
3. Drop the Timestamp column (as it is of no use for our ML modelling work)
4. Examine the distribution of the target classes. If the file is significantly heavy towards the "Benign" class (which is the case with majority of the data files), remove upto 90% of the samples after those "Benign" class samples are stratified across these 2 key features: Protocol and Dst Port. This will ensure that we continue to retain "Benign" class representation under each of those 2 key features even after this data truncation exercise
5. Save the remaining data under the filename suffix of "-staged"


In [99]:
import numpy as np
import pandas as pd
import csv

#First load contents of the file into an array. Do any cleaning of the data along the way
filename_in = 'inputs\\03-02-2018.csv'
lines=[]
header=[]
with open(filename_in, 'r') as in_file:
    reader = csv.reader(in_file)
    i = 0
    for row in reader:
        #Load the header row
        if i == 0:
            header.append(row)
            i = i + 1
        else:
            #Load all other rows while filtering out any re-occurances of the header row
            if str(row).startswith("['Dst Port',")==False:
                lines.append(row)

#Now load that data into a DataFrame
events = pd.DataFrame(lines, columns=header[0])

In [91]:
events.shape

(331100, 80)

In [3]:
events.head(3)

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,0,0,0,0,56320859.5,139.3000358938,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,0,0,0,0,56320733,114.5512985522,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,0,0,0,0,56319311.5,301.9345955667,56319525,56319098,Benign


In [100]:
#Drop columns that are not needed
events.drop(columns='Timestamp', inplace=True)


## Remove excessive 'Benign' samples

In [102]:
#First assess whether we have a highly unbalanced dataset
dist = pd.DataFrame([])
dist['Freq'] = events['Label'].value_counts()
dist['Percentage'] = events['Label'].value_counts(normalize=True)
dist.style.format("{:.3%}", subset=['Percentage'])

,Freq,Percentage
Label,,
Benign,762384,72.707%
Bot,286191,27.293%


In [103]:
#If we have a highly unbalanced dataset, try to remove some samples - with stratification under the Protocol and Dst Port fields 
summary = events[ events['Label'] == 'Benign'][['Protocol', 'Dst Port']]
rowCount = pd.Series(summary.value_counts(), name = 'RecordCount')
mask = rowCount > 200   #Create a mask where any stratified row counts below 400 will be left untouched

In [104]:
rowCount

Protocol  Dst Port
17        53          192898
6         3389        175694
          80          110411
          443          94826
          445          41706
                       ...  
          30143            1
          30146            1
          30147            1
          30150            1
          45250            1
Name: RecordCount, Length: 37334, dtype: int64

In [105]:
rowCount[mask]

Protocol  Dst Port
17        53          192898
6         3389        175694
          80          110411
          443          94826
          445          41706
0         0            13124
17        5355          4578
          3389          1754
          137           1641
          67             707
          123            461
          138            442
6         22             324
Name: RecordCount, dtype: int64

In [106]:
deleteCount = 0
for (protocol, dstport) in rowCount[mask].index.to_list():
    totRows = rowCount[mask][(protocol, dstport)]
    totRowsToDelete = int(0.8 * totRows)
    if (totRows > 400 and totRowsToDelete > 0):
        rowsToDrop = events[ (events['Label'] == 'Benign') & (events['Protocol'] == protocol) & (events['Dst Port'] == dstport)].sample(totRowsToDelete).index
        print(f'About to delete {totRowsToDelete} out of {totRows} within the stratified sample of Protocol = {protocol} and Dst Port = {dstport}')
        events.drop(labels=rowsToDrop, axis=0, inplace=True)
        deleteCount = deleteCount + totRowsToDelete

print(f'Total samples deleted: {deleteCount}')

About to delete 154318 out of 192898 within the stratified sample of Protocol = 17 and Dst Port = 53
About to delete 140555 out of 175694 within the stratified sample of Protocol = 6 and Dst Port = 3389
About to delete 88328 out of 110411 within the stratified sample of Protocol = 6 and Dst Port = 80
About to delete 75860 out of 94826 within the stratified sample of Protocol = 6 and Dst Port = 443
About to delete 33364 out of 41706 within the stratified sample of Protocol = 6 and Dst Port = 445
About to delete 10499 out of 13124 within the stratified sample of Protocol = 0 and Dst Port = 0
About to delete 3662 out of 4578 within the stratified sample of Protocol = 17 and Dst Port = 5355
About to delete 1403 out of 1754 within the stratified sample of Protocol = 17 and Dst Port = 3389
About to delete 1312 out of 1641 within the stratified sample of Protocol = 17 and Dst Port = 137
About to delete 565 out of 707 within the stratified sample of Protocol = 17 and Dst Port = 67
About to del

In [107]:
#Reassess how unbalanced the dataset is now
dist = pd.DataFrame([])
dist['Freq'] = events['Label'].value_counts()
dist['Percentage'] = events['Label'].value_counts(normalize=True)
dist.style.format("{:.3%}", subset=['Percentage'])

,Freq,Percentage
Label,,
Bot,286191,53.197%
Benign,251797,46.803%


In [108]:
#Output to a file that can be saved as a "staged" file
filename_out = filename_in.removesuffix('.csv') + '-staged.csv'
events.to_csv(filename_out, index=False)